In [2]:
import ipywidgets as widgets

bolt_info_layout = widgets.VBox([
    widgets.Label(value="Bolt Info"),

    widgets.HBox([widgets.Label(value="Nominal Size"), widgets.Dropdown(options=["M16", "M20", "M24", "M30", "M36", "M42", "M48", "M56", "M64"], value="M16")]),
    widgets.IntText(value=7, description='Any:', disabled=False)
])

member_info_layout = widgets.VBox([
    widgets.HBox([widgets.Label(value="The $m$ in $E=mc^2$:"), widgets.FloatSlider()]),
    widgets.IntText(value=7, description='Any:', disabled=False)
])


ui = widgets.HBox([
    bolt_info_layout, 
    member_info_layout
])
ui

In [11]:
from properties import BoltMetric
